In [1]:
from pokemon import *
from poke_utils import *
from pokemon_statistics import *
from simulator import runSim, runSimList
from simulator_obj import simulator
import sqlite3
import pickle

In [2]:
%%time
s = simulator('team.txt', ['incineroar', 'rillaboom', 'regieleki', 'amoonguss', 'kyogre','entei'])
q = s.get_output()

Wall time: 4.16 s


In [3]:
%%time
s.pred_lead()

('regieleki', tensor([0.3158, 0.3016, 0.3827], device='cuda:0', grad_fn=<SelectBackward>))
('entei', tensor([0.3532, 0.2814, 0.3654], device='cuda:0', grad_fn=<SelectBackward>))
('amoonguss', tensor([0.2623, 0.3955, 0.3422], device='cuda:0', grad_fn=<SelectBackward>))
('kyogre', tensor([0.2942, 0.3749, 0.3308], device='cuda:0', grad_fn=<SelectBackward>))
('incineroar', tensor([0.3538, 0.3402, 0.3060], device='cuda:0', grad_fn=<SelectBackward>))
('rillaboom', tensor([0.3882, 0.3066, 0.3052], device='cuda:0', grad_fn=<SelectBackward>))
Wall time: 436 ms


('regieleki', 'entei')

In [4]:
# s.parse_battle_history()

The opposing Politoed used Helping Hand!
The opposing Politoed is ready to help the opposing Kingdra!
Rhyperior used Protect!
Rhyperior protected itself!
Porygon2 used Thunderbolt!
(The opposing Kingdra lost 24% of its health!)
The opposing Kingdra fainted!
(Rain continues to fall.)
WonderFluffles sent out Thundurus!


In [5]:
s.my_team_active('Porygon2', 'Slowbro-Galar') 
s.oppo_team_active('incineroar', 'rillaboom') 

print(s.ms.team1.active[0].name)
print(s.ms.team1.active[1].name)
print(s.ms.team1.active)
print(s.ms.team2.active[0].name)
print(s.ms.team2.active[1].name)
print(s.ms.team2.active)

Porygon2
Slowbro-Galar
[<pokemon.pokemon object at 0x00000223C7059880>, <pokemon.pokemon object at 0x00000223C7059B80>]
incineroar
rillaboom
[<pokemon.pokemon object at 0x00000223CBCA7D90>, <pokemon.pokemon object at 0x00000223CBCA7D30>]


In [6]:
%%time
s.findBestMove()


Wall time: 26.8 s


('>p1 move 3 2, move 3 2\n',
 {'move1': '3', 'target1': ' 2', 'move2': '3', 'target2': ' 2'})

In [18]:
1 = 2

SyntaxError: cannot assign to literal (Temp/ipykernel_5284/690182220.py, line 1)

In [16]:
import pickle
f = open("movedex.pkl", "rb")
movedex = pickle.load(f)
f.close()

# Testing below this point

In [ ]:
p1 = createPikalyticsPokemon('incineroar')
p2 = createPikalyticsPokemon('regieleki')

p3 = createPikalyticsPokemon('urshifu')
p4 = createPikalyticsPokemon('grimmsnarl')

combos = genMoveCombos([p1, p2], 'p1', False)
print(len(combos))

combos2 = genMoveCombos([p3, p4], 'p2', False)
print(len(combos2))

print(len(combos) * len(combos2))

combos

In [ ]:
def makeFormatString(poke_name, moves=[], maxmoves=[], ability='', item=''):
    # I should check the poke_name against table names in database or 
    # else ill get sql injected problem here, granted the consequences are small 
    # but lets be honest it also probably isnt
    # put in a validate statement like if(validate(poke_name))
        
    format_string = 'SELECT * FROM ' + poke_name 
    format_tuple = ()
    
    where = False
    
    for move in moves:
        if where:
            format_string += ' AND (move1=? OR move2=? OR move3=? OR move4=?)'
        else:
            format_string += ' WHERE (move1=? OR move2=? OR move3=? OR move4=?)'
            where = True
        format_tuple += (move, move, move, move)
    
    for move in maxmoves:
        if where:
            format_string += ' AND (maxmove1=? OR maxmove2=? OR maxmove3=? OR maxmove4=?)'
        else:
            format_string += ' WHERE (maxmove1=? OR maxmove2=? OR maxmove3=? OR maxmove4=?)'
            where = True
        format_tuple += (move, move, move, move)
                
    if ability_name != '':
        if where: 
            format_string += ' AND ability=?'
        else:
            format_string += ' WHERE ability=?'
            where = True
        format_tuple += (ability_name,)
        
    if item_name != '':
        if where: 
            format_string += ' AND item=?'
        else:
            format_string += ' WHERE item=?'
            where = True
        format_tuple += (item_name,)
    
    
    return format_string, format_tuple

def update_freq(freq, key):
    if key in freq:
        freq[key] += 1
    else:
        freq[key] = 1
        
def update_predictions(pokemon):
    format_string, format_tuple = makeFormatString(pokemon.name, pokemon.moves, pokemon.max_moves, 
                                                   pokemon.ability, pokemon.item)
    cursor.execute(format_string, format_tuple)
    
    total = 0
    move_freq = {}
    ability_freq = {}
    item_freq = {}
    for (num_id, ability, item, move1, move2, move3, move4, _, _, _, _) in cursor:
        total += 1

        update_freq(ability_freq, ability)
        update_freq(item_freq, item)

        update_freq(move_freq, move1)
        update_freq(move_freq, move2)
        update_freq(move_freq, move3)
        update_freq(move_freq, move4)
    
    if "UNKNOWN" in ability_freq.keys():
        ability_freq.pop("UNKNOWN")
    if "UNKNOWN" in item_freq.keys():
        item_freq.pop("UNKNOWN")
    if "UNKNOWN" in move_freq.keys():
        move_freq.pop("UNKNOWN")
    if "UNKNOWN" in maxmove_freq.keys():
        maxmove_freq.pop("UNKNOWN")
    
    if (pokemon.ability == ''):
        key, _ = max(ability_freq.items(), key=lambda x:x[1])
        pokemon.pred_ability = key
    
    if (pokemon.item == ''):
        key, _ = max(item_freq.items(), key=lambda x:x[1])
        pokemon.pred_item = key
    
    pokemon.pred_moves = []
    for i in range(4 - len(pokemon.moves)):
        key, _ = max(move_freq.items(), key=lambda x:x[1])
        pokemon.pred_moves.append(key)
        move_freq.pop(key)
    

In [ ]:
connection = sqlite3.connect("pokemon_profiles.db")
cursor = connection.cursor()